In [ ]:
import itertools
import json
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pygmo
from pymoo.indicators.igd_plus import IGDPlus
from datetime import datetime
import time
import seaborn as sns
import copy
from matplotlib.lines import Line2D

import scenarios as scenariodef
from multiprocess import Pool
# import configurators

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger().setLevel(logging.WARNING)

resultdir = Path("results_full")
figurepath = Path("figures")

In [ ]:
sns.set_style("whitegrid")

configurators = ["RandomSearch", "RandomSearchSI", "RandomSearch25", "default",  "SMAC", "MO-ParamILS",  "ParEGO", "MO-SMAC", "MO-SMAC-PHVI"]

# color_palette = [(230, 159, 0), (86, 180, 233), (0, 158, 115), (240, 228, 66), (0, 114, 178), (213, 94, 0), (204, 121, 167)]  # Wong color palette (works for color-blinded people)
color_palette = [(51, 34, 136), (17, 119, 51), (68, 170, 156), (136, 204, 238), (221, 204, 119), (204, 102, 119), (170, 68, 153), (136, 34, 85),][::-1]  # Tol colorblind palette (https://davidmathlogic.com/colorblind/#%23332288-%23117733-%2344AA99-%2388CCEE-%23DDCC77-%23CC6677-%23AA4499-%23882255)
color_palette = [tuple([float(f"{channel / 255:.3f}") for channel in color]) for color in color_palette]
color_palette += color_palette
# color_palette = sns.color_palette(palette="colorblind")
markers = ["s", "o", "X", "P", "^", "v", ">", "<", "*"][::-1]



def get_style(conf, key):
    return style_guide[conf][key]

# display(style_guide)

config_names = {
    "MO-SMAC-PHVI": "MO-SMAC-PHVI",
    "ParEGO":  "MO-SMAC-PE",
    "MO-SMAC": "MO-SMAC-EHVI",
    "MO-ParamILS": "MO-ParamILS",
    "SMAC": "SMAC",
    "RandomSearchSI": "Random Search + MO-Intensify",
    "RandomSearch": "Random Search",
    "RandomSearch25": "Random Search",
    "default": "Default",
}
reverse_config_names = {v: k for k, v in config_names.items()}

style_guide = {}
for i, conf in enumerate(config_names.values()):
    style_guide[conf] = {"color": color_palette[i%len(color_palette)], "marker": markers[i%len(markers)]}

print(style_guide)

# import matplotlib
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42

# #matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     #"pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     # 'text.usetex': True,
#     # 'pgf.rcfonts': False,
# })

scenario_replacements = {
    "EA_OO_MMMOOP": "EMOA-(hv, sp)",
    "MIP_CPLEX_REGIONS200_cutoff": "MIP-(gap, cutoff)",
    "MIP_CPLEX_REGIONS200_runtime": "MIP-(gap, runtime)",
    "MIP_CPLEX_REGIONS200_CONT_cutoff": "MIP-(gap, cutoff) (c)",  # (c)
    "MIP_CPLEX_REGIONS200_CONT_runtime": "MIP-(gap, runtime) (c)",  # (c)
    "SAT_CMS_QUEENS_runtime_memory": "SAT-(runtime, memory)",
    "SAT_CMS_QUEENS_runtime_solved": "SAT-(runtime, solved)",
    "SAT_ganak_DQMR_runtime_memory": "#SAT-(runtime, solved)",
    "ML_RF_STUDENTS_precision_recall": "ML-(precision, recall)",
    "ML_RF_STUDENTS_precision_recall_size": "ML-(precision, recall, size)",
    "ML_RF_STUDENTS_accuracy_size": "ML-(accuracy, size)",
}

reverse_scenario_replacements = {v: k for k, v in scenario_replacements.items()}

configurator_replacements = config_names

palette = {conf: col for conf, col in zip(config_names.values(), color_palette)}
markers = {conf: col for conf, col in zip(config_names.values(), markers)}
order = configurators  #  [conf for conf in list(config_names.keys()) if conf in list(qdf["configurator"].unique())]


def rename_metadata(df: pd.DataFrame) -> pd.DataFrame:
    for k, v in scenario_replacements.items():
        df.loc[df["scenario"] == k, "scenario"] = v
    for k, v in configurator_replacements.items():
        df.loc[df["configurator"] == k, "configurator"] = v
    return df

print(f"{config_names=}")
print(f"{palette=}")
print(f"{markers=}")

textwidth = 404 * 2.4# pt
pt_per_in = 0.0138888889 
textwidth_in = textwidth * pt_per_in
n_figs_per_row = 4.25
figwidth = textwidth_in / n_figs_per_row
figsize = (figwidth, figwidth * 0.8)

# Performance results
Configuration procedure:
- Run a configurator $n$ times on a scenario with a different random seed.
- Validate the final incumbent (single or multiple configurations) of each configuration run on a common validation set (subset of the train set).
- Combine all the incumbents into one archive and throw out all the dominated configurations. This configuration in the archive are non-dominated.
- Validate configurations in the filtered archive on the training set.

HV computation:
- Determine reference point by choosing the largest obtained mean value for an objective by all validation and training runs of all configurations found by all configurators.
- Compute the HV.

## Dataloading

In [ ]:
scenario_files = [d for d in resultdir.joinpath("collecttraj").iterdir()]
scenarios = []
for i, s in enumerate(scenario_files):
    print(Path(s).name)
    dfpath = f"{resultdir}/collecttraj/dataframes/{Path(s).name}.csv"
    metapath = f"{resultdir}/collecttraj/metadata/{Path(s).name}.json"
    finalincpicklepath = f"{resultdir}/collect/{Path(s).name}"     

    loaded_from_csv: bool = False
    try:
        with open(s, "rb") as fh:
            scenario = pickle.load(fh)
    except:
        # try json and csv
        if not (Path(dfpath).exists() and Path(metapath).exists()):
            print(f"Couldn't load {s}")
            continue

        df = pd.read_csv(dfpath)
        loaded_from_csv = True
        with open(metapath, "r") as f:
            scenario = json.load(f)

        scenario["run_result"] = df
        pass
    
    scenario_name = str(scenario['experiment']["scenarios"])
    scen = getattr(scenariodef, scenario_name)()
    objectives = [o["name"] for o in scen.objectives]

    if scenario["experiment"]["scenarios"] in ["MIP_CPLEX_REGIONS200_CONT_cutoff",
                                               "MIP_CPLEX_REGIONS200_CONT_runtime",
                                               "SAT_CMS_QUEENS_runtime_memory",
                                               "SAT_ganak_DQMR_runtime_memory",
                                               "ML_RF_STUDENTS_precision_recall",
                                               "ML_RF_STUDENTS_precision_recall_size",
                                               "ML_RF_STUDENTS_accuracy_size",
                                               "EA_OO_MMMOOP"
                                               ]:
        df = scenario["run_result"]
        print(f"{df['configurators'].unique()=}")
        df = df[~df["configurators"].isin(["MO-SMAC", "RandomSearchSI","RandomSearch"])]
        
        #Load final incumbent and load trajectories
        with open(finalincpicklepath, "rb") as fh:
            finalincresults = pickle.load(fh)
        fdf = finalincresults["run_result"]
        fdf = fdf[~fdf["configurators"].isin(["MO-SMAC", "RandomSearchSI","RandomSearch"])]
        fdf["seeds"] = fdf["seeds"].astype(int)
        fdf = fdf[fdf["seeds"] <  20]
        keys = ["scenarios", "configurators", "seeds"]
        for crun, grp in fdf.groupby(keys):
            keyvals = crun
            crun = list(crun)
            crun[0] = crun[0].replace("_","-")
            configurepath = resultdir / "configure" / ("_".join([str(c) for c in crun])+".pickle")
            if not configurepath.exists():
                continue
            
            with open(configurepath, "rb") as fh:
                configdata = pickle.load(fh)
            
            if keyvals[1] == "SMAC":
                last_incumbent = [traj["trajectory"][-1]["config_ids"][0] for traj in configdata["run_result"]["trajectory"]]
                actual_config_ids = {str(k): f"{k}_{v}" for k, v in enumerate(last_incumbent)}
            elif keyvals[1] == "default":
                last_incumbent = configdata["run_result"]["trajectory"][-1]["config_ids"]
                actual_config_ids = {str(k): str(v) for k, v in enumerate(last_incumbent)} 
            else:
                last_incumbent = configdata["run_result"]["trajectory"]["trajectory"][-1]["config_ids"]
                actual_config_ids = {str(k): str(v) for k, v in enumerate(last_incumbent)} 
            
            mask = [fdf[k] == v for k, v in zip(keys, keyvals)]
            mask = [all(row) for row in zip(*mask)]
            fdf.loc[mask, "configuration"] = fdf.loc[mask, "configuration"].replace(actual_config_ids)

        df["seeds"] = df["seeds"].astype(int)
        print("seeds", list(df["seeds"].unique()))

        df = df[df["seeds"] <  20]
        
        print(f"{len(df)=}")
        df = pd.concat([df, fdf], ignore_index=True)
        print(f"{len(df)=}")
        
        df["action"] = df["action"].replace({"testtraj": "test", "validatetraj":"validate"})
        df["configid"] = [(r["seeds"], r["configuration"]) for _,r in df.iterrows()]
        df.loc[df["status"] == "CRASHED", objectives] = df[df["status"] != "CRASHED"][objectives].max().to_numpy()
        
        scenario_name = str(scenario['experiment']["scenarios"])
        scen = getattr(scenariodef, scenario_name)()
        objectives = [o["name"] for o in scen.objectives]
        scenario["objectives"] = objectives
        df = df.groupby(["scenarios", "action", "seeds", "configurators", "configuration", "configid"])[objectives].mean().reset_index()
        print(f"{df[df.isna().any(axis=1)]=}")
        print(f"{len(df)=}")
        
        #Normalize and compute reference set
        stats_dest = Path(f"intermediates/norm_stats/{scenario_name}.csv")
        if stats_dest.exists():
            stats = pd.read_csv(stats_dest, index_col=[0, 1])
            print(f"Got normalisation stats from file {stats_dest}")
        else:
            stats = df.groupby("action")[objectives].describe().stack()

        reference_set = dict()
        igdp = dict()
        
        #Normalize objectives
        for a in ["test", "validate"]:
            for o in objectives:
                df.loc[df["action"] == a, o+"_norm"] = 1 + ((df.loc[df["action"] == a, o] - stats.loc[(a,"min"), o]) / (stats.loc[(a,"max"), o] - stats.loc[(a,"min"), o])).clip(0, 1)
                
            points = df.loc[df["action"] == a, [o+"_norm" for o in objectives]].to_numpy()
            refset_ids = pygmo.fast_non_dominated_sorting(points)[0][0]
            refset = points[refset_ids, :]
            reference_set[a] = refset.tolist()
        
        scenario["reference_set"] = reference_set
        scenario["reference_point"] = [2.0+1e-1] * len(objectives)
        
        scenario["run_result"] = df
        scenarios.append(scenario)

        #export
        for p in [dfpath, metapath]:
            Path(p).parent.mkdir(parents=True, exist_ok=True )
        df = scenario["run_result"]
        if not loaded_from_csv:
            df.to_csv(dfpath, index=False)
        meta_data = copy.copy(scenario)
        del meta_data["run_result"]
        with open(metapath, "w") as f:
            json.dump(meta_data, f)

print(f"Got {len(scenarios)} scenarios")

for scenario in scenarios:
    for k, v in scenario_replacements.items():
        scenario["run_result"].loc[scenario["run_result"]["scenarios"] == k, "scenarios"] = v
    for k, v in configurator_replacements.items():
        scenario["run_result"].loc[scenario["run_result"]["configurators"] == k, "configurators"] = v
    

scenarios = sorted(scenarios, key=lambda s: s["experiment"]["scenarios"])  # Sort by name
[s["experiment"]["scenarios"] for i, s in enumerate(scenarios)]

In [ ]:
# df[df["configurators"] == "SMAC"]
df["configurators"].unique()

# Trajectory plots

## Metric calculation

In [ ]:
# Helper functions
def compute_hypervolume(configurations, df: pd.DataFrame, objectives, reference_point, action="validate") -> float:
    df = df[df["action"] == action]
    points = df[df["configid"].isin(configurations)][objectives].to_numpy()
    return pygmo.hypervolume(points).compute(reference_point)

def compute_igdp(configurations, df: pd.DataFrame, objectives, refset, action="validate"):
    igdp = IGDPlus(np.array(refset))
    df = df[df["action"] == action]
    points = df[df["configid"].isin(configurations)][objectives].to_numpy()
    return igdp(points)

def get_nd_configurations(configurations: list[tuple], df, objectives, action="validate") -> list[int]:
    # Get performance on validate set
    df = df[df["action"] == action]
    df = df[df["configid"].isin(configurations)]
    points = df[objectives].to_numpy()
    configids = list(df["configid"])
    
    if len(points) == 1:
        return configids
    else:
        ndp = pygmo.fast_non_dominated_sorting(points)[0][0]
        return [configids[i] for i in ndp]

## Trajectory gathering

In [ ]:
def compute_meta_trajectory(trajectories: dict[int, dict]) -> dict:
    """
    Compute the trajectory for from multiple configuration run trajectories.
    """
    
    meta_trajectory = {}
    last_event = None
    events = [[k for k, _ in t.items()] for _, t in trajectories.items()]
    traj_keys = [k for k in trajectories.keys()]
    pointers = np.zeros(len(events), dtype=int)
    
    while np.count_nonzero(pointers >= 0) > 0:
        # Find next trajectory change
        next_change = np.argmin([events[s][k] if k >= 0 else np.inf for s, k in enumerate(pointers)])
        event_time = events[next_change][pointers[next_change]]
        
        new_state = [] if last_event is None else copy.copy(meta_trajectory[last_event])
        
        # Remove old state from the respective trajectory run
        new_state = [c for c in new_state if c[0] != traj_keys[next_change]]
        
        # Add new state from the respective trajectory run   
        configurations = trajectories[traj_keys[next_change]][event_time]
        new_state += [(traj_keys[next_change], c) for c in configurations]

        meta_trajectory[event_time] = new_state

        # Update pointer
        pointers[next_change] += 1
        if pointers[next_change] == len(events[next_change]):
            pointers[next_change] = -1
            
        # Update last time
        last_event = event_time
        
    return meta_trajectory


def get_trajectory_from_file(scenario, configurator, seed):
    scenario = reverse_scenario_replacements[scenario]
    scenario = scenario
    
    configurator = reverse_config_names[configurator]
    
    configurepath = resultdir / "configure" / f"{scenario.replace('_','-')}_{configurator}_{seed}.pickle"
    if not configurepath.exists():
        print(f" {configurepath} does not exist")
        return
    
    with open(configurepath, "rb") as fh:
        configdata = pickle.load(fh)

    if configurator == "SMAC":
        
        #SO procedure cutoff needs to be adjusted to represent the full budget again.
        scen = getattr(scenariodef, scenario)()
        if "MIP" in scenario:
            corr_factor = [sum(scen.cutoffs)/c for c in scen.cutoffs]
        else:
            corr_factor = [len(scen.objectives)]*len(scen.objectives)
        
        trajectories = {}
        for i, traj in enumerate(configdata["run_result"]["trajectory"]):
            if i >= len(corr_factor):
                # print(f"More runs found than objectives in {scenario} for {configurator}")
                # REMOVES F1 from ML scenarios
                break
            trajectory = traj["trajectory"]
            trajectory = {t["walltime"]*corr_factor[i]: [f"{i}_{c}" for c in t["config_ids"]] for t in trajectory}
            trajectories[i] = trajectory
            
        trajectory = compute_meta_trajectory(trajectories)
        trajectory = {k: [c for _, c in v] for k, v in trajectory.items()}
    elif configurator == "default":
        trajectory = configdata["run_result"]["trajectory"]
        trajectory = {t["walltime"]: [str(c) for c in t["config_ids"]] for t in trajectory}
    else:
        trajectory = configdata["run_result"]["trajectory"]["trajectory"]
        trajectory = {t["walltime"]: [str(c) for c in t["config_ids"]] for t in trajectory}
    return trajectory

def get_all_trajectories_from_file(scenario, configurator):
    trajectories = {}
    for seed in range(20):
        trajectories[seed] = get_trajectory_from_file(scenario, configurator, seed)
    return trajectories

In [ ]:
scenario = scenarios[0]
scenario_name = str(scenario['experiment']["scenarios"])
print(scenario_name)
scen = getattr(scenariodef, scenario_name)()
objectives = [o["name"] for o in scen.objectives]
print(objectives)
df = scenario["run_result"]

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
get_trajectory_from_file('MIP-(gap, cutoff) (c)', 'SMAC', 0)

In [ ]:
get_nd_configurations([(11, '0_1')], df, objectives)

In [ ]:
# Compute reference data
# Reference nadir point
# Reference Pareto set
# Normalisation bounds per objectives

# stats = df.groupby("action")[objectives].describe().stack()
# 
# reference_set = dict()
# igdp = dict()
# 
# #Normalize objectives
# for a in ["test", "validate"]:
#     for o in objectives:
#         df.loc[df["action"] == a, o+"_norm"] = 1 + ((df.loc[df["action"] == a, o] - stats.loc[(a,"min"), o]) / (stats.loc[(a,"max"), o] - stats.loc[(a,"min"), o]))
#         
#     points = df.loc[df["action"] == a, [o+"_norm" for o in objectives]].to_numpy()
#     refset_ids = pygmo.fast_non_dominated_sorting(points)[0][0]
#     refset = points[refset_ids, :]
#     reference_set[a] = refset
#     igdp[a] = IGDPlus(refset)
#         
# reference_point = [2.0+1e-1]*len(objectives)
# 
# reference_set

In [ ]:
from joblib import Parallel, delayed

In [ ]:
for scenario_id, scenario in enumerate(scenarios):
    scenario_name = str(scenario['experiment']["scenarios"])
    print(scenario_name)
    scen = getattr(scenariodef, scenario_name)()
    objectives = scenario["objectives"]
    objectivesnorm = [o+"_norm" for o in objectives]
    print(objectives)
    
    # trajdfpath = Path(f"intermediates/sampled_trajectories/8runs_190samples_{scenario_name}.csv")
    trajdfpath = Path(f"intermediates/sampled_trajectories/new_8runs_190samples_{scenario_name}.csv")
    
    if trajdfpath.exists():
        print("Getting samples from file")
        trajdf = pd.read_csv(str(trajdfpath))
        trajdf = trajdf[trajdf["configurator"] != "Random Search"]
        trajdf["configurator"].replace("Random Search 25", "Random Search")
        scenarios[scenario_id]["trajectorydf"] = trajdf
        continue
    
    df = scenario["run_result"]
    keys = ["scenarios", "configurators"]
    trajdf = []
    # for crun, grp in df.groupby(keys):
    #     print(crun)
    #     # Get the trajectory from the configuration run
    #     keyvals = crun
    #     trajectories = get_all_trajectories_from_file(*crun)
    #     rng = np.random.RandomState(42)
    #     for i in tqdm(range(190)):
    #         selected_seeds = rng.choice(20, 8, replace=False)
    #         trajectory = compute_meta_trajectory({k: v for k,v in trajectories.items() if k in selected_seeds})
    #         for wtime, perf in trajectory.items():
    #             perf = get_nd_configurations(perf, df, objectives)
    #             result_row = {
    #                 "configurator": crun[1],
    #                 "permutation": i,
    #                 "walltime": wtime,
    #                 "hv_val": compute_hypervolume(perf, df, objectivesnorm, scenario["reference_point"], action="validate"),
    #                 "hv_test": compute_hypervolume(perf, df, objectivesnorm, scenario["reference_point"], action="test"),
    #                 "igdp_val": compute_igdp(perf, df, objectivesnorm, scenario["reference_set"]["validate"], action="validate"),
    #                 "igdp_test": compute_igdp(perf, df, objectivesnorm, scenario["reference_set"]["test"], action="test"),
    #             }
    #             trajdf.append(result_row)
        
    def process_permutation(i, crun, trajectories, df, objectives, objectivesnorm, scenario):
        rng = np.random.RandomState(i)
        selected_seeds = rng.choice(20, 8, replace=False)
        trajectory = compute_meta_trajectory({k: v for k, v in trajectories.items() if k in selected_seeds})
        local_results = []
        for wtime, perf in trajectory.items():
            perf = get_nd_configurations(perf, df, objectives)
            result_row = {
                "configurator": crun[1],
                "permutation": i,
                "walltime": wtime,
                "hv_val": compute_hypervolume(perf, df, objectivesnorm, scenario["reference_point"], action="validate"),
                "hv_test": compute_hypervolume(perf, df, objectivesnorm, scenario["reference_point"], action="test"),
                "igdp_val": compute_igdp(perf, df, objectivesnorm, scenario["reference_set"]["validate"], action="validate"),
                "igdp_test": compute_igdp(perf, df, objectivesnorm, scenario["reference_set"]["test"], action="test"),
            }
            local_results.append(result_row)
        return local_results
    
    trajdf = []
    for crun, grp in df.groupby(keys):
        print(crun)
        # Get the trajectory from the configuration run
        keyvals = crun
        trajectories = get_all_trajectories_from_file(*crun)
    
        num_permutations = 190
        results = Parallel(n_jobs=6, )(
            delayed(process_permutation)(i, crun, trajectories, df, objectives, objectivesnorm, scenario) for i in tqdm(range(num_permutations))
        )
    
        trajdf += [row for result in results for row in result]
        
    trajdf = pd.DataFrame(trajdf)
    
    trajdfpath.parent.mkdir(exist_ok=True, parents=True)
    trajdf.to_csv(str(trajdfpath))
    scenarios[scenario_id]["trajectorydf"] = trajdf

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
palette["Random Search 25"] = palette["Random Search"]
palette["Random Search (25)"] = palette["Random Search"]
palette

In [ ]:
sns.set(font_scale=1.)
for scenario_id, scenario in enumerate(scenarios):
    scenario_name = str(scenario['experiment']["scenarios"])
    print(scenario_name)
    scen = getattr(scenariodef, scenario_name)()
    objectives = scenario["objectives"]
    objectivesnorm = [o+"_norm" for o in objectives]
    print(objectives)
    hvdf = scenario["trajectorydf"]
    
    flattened = []
    
    for timestep in tqdm(np.linspace(0, hvdf["walltime"].max(), 128, dtype=int)):
    # for timestep in tqdm(hvdf["walltime"].astype(int).sort_values().unique()):  # point at every change
        current_idx = hvdf[hvdf["walltime"] <= timestep].groupby(["configurator", "permutation"])["walltime"].idxmax()
        fdf = copy.copy(hvdf.loc[current_idx].reset_index())
        fdf["walltime"] = timestep
        flattened.append(fdf)
            
    flattened = pd.concat(flattened, ignore_index=True)
        
    for o in itertools.product(["hv", "igdp"], ["val", "test"]):
        print(o)
        target = "_".join(o)
        fig, ax = plt.subplots(1, 1, figsize=figsize, dpi=300)
        g = sns.lineplot(data=flattened, x="walltime", y=target, hue="configurator", palette=palette, ax=ax)
        # g._legend.remove()
        ax.get_legend().remove()
        
        # last_point = flattened.iloc[flattened.groupby("configurator")["walltime"].idxmax()]
        # ax.scatter(last_point["walltime"], last_point[target])
        
        # fig, ax = plt.subplots(1, 1, figsize=(4,3))
        # for configurator, grp in flattened.groupby("configurator"):
        #     grp = grp.sort_values("walltime")
        #     x = grp["walltime"].unique()
        #     mean = grp.groupby("walltime")[target].mean()
        #     ax.plot(x, mean, label=configurator, color=style_guide[configurator]["color"], zorder=10)
        #     std = grp.groupby("walltime")[target].std()
        #     ax.fill_between(x, mean - 0.25*std, mean + 0.25*std, alpha=0.25, color=style_guide[configurator]["color"], zorder=-10)
        
        limits = flattened.groupby(["walltime","configurator"])[target].mean().reset_index()
        limits = limits[limits["walltime"] > limits["walltime"].quantile(0.1)][target]
        limits = (limits.min()*0.98, limits.max()*1.02)
        print(f"{limits}")
        ax.set_ylim(limits)
        ax.set_xlabel("Walltime [seconds]")
        metric_names={"hv": "Hypervolume", "igdp": "IGD+"}
        partition_names={"val": "validation", "test": "test"}
        ax.set_ylabel(f"{metric_names[o[0]]} ({partition_names[o[1]]})")
        ax.set_ylabel(f"{metric_names[o[0]]}")
        ax.set_title(scenario_replacements[scenario_name].replace("(c)",""))
        figurepathfile = Path(figurepath / f"trajectories/{target}_{scenario_name}.pdf")
        figurepathfile.parent.mkdir(parents=True, exist_ok=True)
        # if o[0] == "hv":
        #     plt.ylim(1, 1.15)
        # if o[0] == "igdp":
        #     plt.ylim(0, 0.05)
        plt.tight_layout()
        plt.savefig(str(figurepathfile), bbox_inches='tight', pad_inches=0)
        plt.clf()
        # plt.show()

In [ ]:
# flattened[flattened["walltime"] >= flattened["walltime"].quantile(0.5)]["hv_val"].describe()
flattened.iloc[flattened.groupby("configurator")["walltime"].idxmax()]

In [ ]:
for o in itertools.product(["hv", "igdp"], ["val", "test"]):
    print(o)
    target = "_".join(o)
    # sns.lineplot(data=flattened, x="walltime", y=target, hue="configurator")
    for configurator, grp in flattened.groupby("configurator"):
        grp = grp.sort_values("walltime")
        x = grp["walltime"].unique()
        plt.plot(x, grp.groupby("walltime")[target].mean(), label=configurator, color=style_guide[configurator]["color"], zorder=10)
        plt.fill_between(x, grp.groupby("walltime")[target].quantile(0.05), grp.groupby("walltime")[target].quantile(0.95), alpha=0.25, color=style_guide[configurator]["color"], zorder=-10)
    plt.title(scenario_name+" (runs=8)")
    limits = (flattened[target].quantile(0.05)*0.98, flattened[target].quantile(0.95)*1.02)
    plt.ylim(limits)
    plt.legend()
    # figurepath = Path(figurepath / f"trajectories/{target}_{scenario_name}.pdf")
    # figurepath.parent.mkdir(parents=True, exist_ok=True)
    # plt.savefig(str(figurepath))
    plt.show()